# ・ライブラリのインポート

In [2]:
import pandas as pd
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import warnings
# I expect to see RuntimeWarnings in this block
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=RuntimeWarning)
    foo = np.nanmean(score_mean, axis=0)

from xgboost import XGBClassifier

import itertools
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

NameError: name 'score_mean' is not defined

# ・データの読み込み

In [3]:
train = pd.read_csv('/Users/markun/SIGNATE/train.csv')
test = pd.read_csv('/Users/markun/SIGNATE/test.csv')

In [20]:
test

,index,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,398,0,126,80,0,0,40.096264,0.822517,21
1,3833,3,88,60,20,0,39.810590,0.204331,22
2,4836,3,114,76,0,0,33.198760,0.521011,21
3,4572,1,146,74,0,0,26.890259,0.504950,38
4,636,1,123,90,26,140,40.270088,0.800513,28
...,...,...,...,...,...,...,...,...,...
1995,3138,4,150,60,0,0,39.385785,0.175051,26
1996,191,6,153,88,0,0,38.614204,0.509367,28
1997,3294,7,106,78,0,0,51.678147,0.728404,29
1998,3073,2,101,70,0,0,40.271989,0.316558,26


In [21]:
train

,index,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,200,9,125,74,0,0,28.536910,0.444902,45,1
1,3832,4,109,80,0,0,28.047673,0.238243,22,0
2,4927,4,88,78,39,0,52.371341,0.279471,26,0
3,4088,9,125,74,0,0,40.062688,0.203922,45,0
4,3644,5,107,78,44,284,52.935068,0.284959,45,1
...,...,...,...,...,...,...,...,...,...,...
2995,4931,4,88,74,17,0,33.848723,0.171073,23,0
2996,3264,0,144,88,0,0,26.846832,0.259957,21,1
2997,1653,6,117,96,36,0,28.101646,0.716126,22,1
2998,2607,2,113,74,0,0,33.079021,0.266179,38,1


# ・目的変数を除外

In [1]:
train_x = train.drop(['Outcome','Insulin'], axis = 1)
train_x 

NameError: name 'train' is not defined

# ・目的変数

In [23]:
train_y = train['Outcome']
train_y

0       1
1       0
2       0
3       0
4       1
       ..
2995    0
2996    1
2997    1
2998    1
2999    0
Name: Outcome, Length: 3000, dtype: int64

# ・学習及び検証

In [24]:
xgb_params_space = {
    'objective':'binary:logistic',
    'max_depth': [3,4,5,6,7,8,9], 
    'eta': 0.1,
    'eval_metric':'logloss',
    'colsample_bytree':[0.8,0.85,0.9,0.95,1.0],
    'subsample':0.8,
    'gamma':0,
    'lambda':1,
    'alpha':0,
    'min_child_weight':[0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0]
}

param_combinations = itertools.product(xgb_params_space['max_depth'], xgb_params_space['min_child_weight'], xgb_params_space['colsample_bytree'])

params = []
scores = []
scores_logloss = []
scores_accuracy = []

for max_depth, min_child_weight, colsample_bytree in param_combinations:
    scores_folds = []
    
    #KFold
    kf = KFold(n_splits = 10, shuffle = True, random_state = 55)
    for tr_idx, va_idx in kf.split(train_x):
        tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
        tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
        
        #学習
        model = XGBClassifier(n_estimators = 20, random_state = 66,
                              max_depth = max_depth, min_child_weight = min_child_weight, colsample_bytree = colsample_bytree )
        model.fit(tr_x, tr_y)

        #予測、損失関数、精度の計算
        va_pred = model.predict_proba(va_x)[:, 1]
        logloss = log_loss(va_y, va_pred)
        accuracy = accuracy_score(va_y, va_pred > 0.5)
    
    score_mean = np.mean(scores_folds)
    
    params.append((max_depth, min_child_weight, colsample_bytree))
    scores.append(score_mean)
    
best_idx = np.argsort(scores)[0]
best_param = params[best_idx]
print(f'max_depth: {best_param[0]}, min_child_weight: {best_param[1]}, colsample_bytree:{best_param[2]}')


/Users/markun/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/markun/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/markun/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/markun/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/markun/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/markun/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/markun/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.

KeyboardInterrupt: 

# ・XGB

In [ ]:
xgb_params_space = {
    'objective':'binary:logistic',
    'max_depth': [3,4,5,6,7,8,9], 
    'eta': [0.1],
    'eval_metric':'logloss',
    'colsample_bytree':[0.8,0.85,0.9,0.95,1.0],
    'subsample':0.8,
    'gamma':0,
    'lambda':1,
    'alpha':0,
    'min_child_weight':[0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0]
}

param_conbinations = itertools.product(xgb_params_space['max_depth'], xgb_params_space['min_child_weight'])

params = []
scores = []

for max_depth, min_child_weight in param_combinations:
    scores_folds = []

#  ・テストデータに対する予測

In [ ]:
prediction = bst.predict(xgb.DMatrix(test))
prediction = np.where(prediction < 0.5, 0,1)
prediction

# ・提出用ファイルの作成

In [ ]:
submission6 = pd.DataFrame({
    'index':test['index'],
    'Outcome':prediction, 
})
submission6.to_csv('/Users/markun/SIGNATE/submission6.csv', header = False, index = False )

In [ ]:
submission5.head()